In [137]:
import numpy as np
import pandas as pd
from igraph import *
import networkx as nx

In [138]:
udf = pd.read_csv('https://raw.githubusercontent.com/purplesmurf45/Migration-visualisation/main/data/updated_refugees_without_unknown.csv')

In [139]:
print(udf.columns)
udf

Index(['Year', 'Origin', 'Destination', 'Refugees'], dtype='object')


,Year,Origin,Destination,Refugees
0,2000,Iraq,Albania,9
1,2000,Serbia and Kosovo: S/RES/1244 (1999),Albania,507
2,2000,Türkiye,Albania,5
3,2000,Chad,Algeria,20
4,2000,Cameroon,Algeria,5
...,...,...,...,...
75501,2021,Rwanda,Zimbabwe,660
75502,2021,Somalia,Zimbabwe,13
75503,2021,Sudan,Zimbabwe,8
75504,2021,South Sudan,Zimbabwe,8


In [140]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
le = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
udf['node_dest']= le.fit_transform(udf['Destination'])
udf['node_ori']= le.fit_transform(udf['Origin'])
udf['Refugees'] = udf['Refugees'].replace(np.nan, 0)

In [141]:
udf

,Year,Origin,Destination,Refugees,node_dest,node_ori
0,2000,Iraq,Albania,9,1,84
1,2000,Serbia and Kosovo: S/RES/1244 (1999),Albania,507,1,156
2,2000,Türkiye,Albania,5,1,184
3,2000,Chad,Algeria,20,2,34
4,2000,Cameroon,Algeria,5,2,30
...,...,...,...,...,...,...
75501,2021,Rwanda,Zimbabwe,660,178,147
75502,2021,Somalia,Zimbabwe,13,178,163
75503,2021,Sudan,Zimbabwe,8,178,168
75504,2021,South Sudan,Zimbabwe,8,178,165


In [142]:
print(udf.isnull().sum())
print(len(pd.concat([udf["node_dest"],udf["node_ori"]]).unique()))
print(len(udf["node_dest"].unique()))
print(len(udf["node_ori"].unique()))

Year           0
Origin         0
Destination    0
Refugees       0
node_dest      0
node_ori       0
dtype: int64
199
179
199


In [143]:
udf

,Year,Origin,Destination,Refugees,node_dest,node_ori
0,2000,Iraq,Albania,9,1,84
1,2000,Serbia and Kosovo: S/RES/1244 (1999),Albania,507,1,156
2,2000,Türkiye,Albania,5,1,184
3,2000,Chad,Algeria,20,2,34
4,2000,Cameroon,Algeria,5,2,30
...,...,...,...,...,...,...
75501,2021,Rwanda,Zimbabwe,660,178,147
75502,2021,Somalia,Zimbabwe,13,178,163
75503,2021,Sudan,Zimbabwe,8,178,168
75504,2021,South Sudan,Zimbabwe,8,178,165


In [144]:
node_set = 199

In [145]:
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

In [146]:
zero_list = [11, 22, 24, 32, 46, 51, 53, 64, 65, 69, 80, 85, 89, 93, 104, 105, 110, 115, 119, 129, 131, 135, 136, 142, 143, 149, 152, 157, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198]

In [147]:
len(zero_list)

49

In [148]:
filtered = udf[~udf['node_dest'].isin(zero_list)]
filtered = filtered[~filtered['node_ori'].isin(zero_list)]
filtered

,Year,Origin,Destination,Refugees,node_dest,node_ori
0,2000,Iraq,Albania,9,1,84
1,2000,Serbia and Kosovo: S/RES/1244 (1999),Albania,507,1,156
3,2000,Chad,Algeria,20,2,34
4,2000,Cameroon,Algeria,5,2,30
5,2000,Congo,Algeria,30,2,41
...,...,...,...,...,...,...
75487,2021,Sierra Leone,Zambia,5,177,158
75488,2021,Somalia,Zambia,3540,177,163
75489,2021,Sudan,Zambia,29,177,168
75490,2021,South Sudan,Zambia,5,177,165


In [149]:
print(filtered.isnull().sum())
print(len(pd.concat([filtered["node_dest"],filtered["node_ori"]]).unique()))
print(len(filtered["node_dest"].unique()))
print(len(filtered["node_ori"].unique()))

Year           0
Origin         0
Destination    0
Refugees       0
node_dest      0
node_ori       0
dtype: int64
150
150
150


In [150]:
fil_wo = filtered
fil_wo = fil_wo.drop(['node_dest','node_ori'], axis = 1)
fil_wo

,Year,Origin,Destination,Refugees
0,2000,Iraq,Albania,9
1,2000,Serbia and Kosovo: S/RES/1244 (1999),Albania,507
3,2000,Chad,Algeria,20
4,2000,Cameroon,Algeria,5
5,2000,Congo,Algeria,30
...,...,...,...,...
75487,2021,Sierra Leone,Zambia,5
75488,2021,Somalia,Zambia,3540
75489,2021,Sudan,Zambia,29
75490,2021,South Sudan,Zambia,5


In [151]:
# label_encoder object knows how to understand word labels.
le1 = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
fil_wo['node_dest']= le1.fit_transform(fil_wo['Destination'])
fil_wo['node_ori']= le1.fit_transform(fil_wo['Origin'])
fil_wo

,Year,Origin,Destination,Refugees,node_dest,node_ori
0,2000,Iraq,Albania,9,1,73
1,2000,Serbia and Kosovo: S/RES/1244 (1999),Albania,507,1,129
3,2000,Chad,Algeria,20,2,30
4,2000,Cameroon,Algeria,5,2,27
5,2000,Congo,Algeria,30,2,37
...,...,...,...,...,...,...
75487,2021,Sierra Leone,Zambia,5,149,130
75488,2021,Somalia,Zambia,3540,149,135
75489,2021,Sudan,Zambia,29,149,140
75490,2021,South Sudan,Zambia,5,149,137


In [152]:
print(fil_wo.isnull().sum())
print(len(pd.concat([fil_wo["node_dest"],fil_wo["node_ori"]]).unique()))
print(len(fil_wo["node_dest"].unique()))
print(len(fil_wo["node_ori"].unique()))
print(fil_wo["node_ori"].max())

Year           0
Origin         0
Destination    0
Refugees       0
node_dest      0
node_ori       0
dtype: int64
150
150
150
149


In [153]:
req_cols = ['node_dest', 'node_ori', 'Refugees']
X = udf[udf['Year'] == 2016][req_cols]
Y = filtered[filtered['Year'] == 2016][req_cols]
Z = fil_wo[fil_wo['Year'] == 2016][req_cols]

In [154]:
print(X)
print(Y)
print(Z)

       node_dest  node_ori  Refugees
51027          0        83        33
51028          0       133     59737
51029          1        56         5
51030          1        36        11
51031          1        50         5
...          ...       ...       ...
54884         41        39        35
54885         41        45         6
54886         41        88         7
54887         41       193         5
54888        143        83         5

[3862 rows x 3 columns]
       node_dest  node_ori  Refugees
51027          0        83        33
51028          0       133     59737
51029          1        56         5
51030          1        36        11
51031          1        50         5
...          ...       ...       ...
54869        177       168        37
54870        177       172         5
54884         41        39        35
54885         41        45         6
54886         41        88         7

[3219 rows x 3 columns]
       node_dest  node_ori  Refugees
51027          0        7

In [155]:
def get_nx_graph(df, year, num_nodes):
    g = Graph(directed=True)
#     num_nodes = len(pd.concat([df["node_dest"],df["node_ori"]]).unique())
    g.add_vertices(num_nodes)
    dfx = df[df['Year'] == year]
    # Add ids and labels to vertices
    for i in range(len(g.vs)):
        g.vs[i]["id"]= i
        g.vs[i]["label"]= str(i)
    # Add edges
    for i in list(dfx.index):
        g.add_edge(dfx['node_ori'][i], dfx['node_dest'][i], weight = dfx['Refugees'][i], label= dfx['Refugees'][i])
    G = g.to_networkx()
    return G

In [156]:
G1 = get_nx_graph(udf, 2016, 199)
G2 = get_nx_graph(filtered, 2016, 199)
G3 = get_nx_graph(fil_wo, 2016, 150)

In [157]:
adj_mat1 = nx.to_numpy_array(G1)
adj_mat2 = nx.to_numpy_array(G2)
adj_mat3 = nx.to_numpy_array(G3)
print(adj_mat1.shape)
print(adj_mat2.shape)
print(adj_mat3.shape)

(199, 199)
(199, 199)
(150, 150)


In [158]:
def print_freq_table(array):
    unique, counts = np.unique(array, return_counts=True)
    print(np.asarray((unique, counts)).T)

# Original Algorithms

In [159]:
def get_communities(df, year, cd, weighted):
    g = Graph(directed=True)
    node_set = len(pd.concat([df["node_dest"],df["node_ori"]]).unique())
    print(node_set)
    g.add_vertices(node_set)
    dfx = df[df['Year'] == year]
    # Add ids and labels to vertices
    for i in range(len(g.vs)):
        g.vs[i]["id"]= i
        g.vs[i]["label"]= str(i)
    # Add edges
    for i in list(dfx.index):
        g.add_edge(dfx['node_ori'][i], dfx['node_dest'][i], weight = dfx['Refugees'][i], label= dfx['Refugees'][i])
    if cd == 'info':
        if weighted == 1:
            return g.community_infomap(edge_weights='weight')
        else:
            return g.community_infomap()
    elif cd == 'label':
        if weighted == 1:
            return g.community_label_propagation(weights=g.es['weight'])
        else:
            return g.community_label_propagation()
    elif cd == 'modn':
        if weighted == 1:
            return g.community_leiden(weights=g.es['weight'])
        else:
            return g.community_leiden()

In [160]:
labels_info1 = get_communities(udf, 2016,'info', 1)
# labels_info2 = get_communities(filtered, 2016,'info', 1)
labels_info3 = get_communities(fil_wo, 2016,'info', 1)

199
150


In [161]:
labels_slpa1 = get_communities(udf, 2016,'label', 1)
# labels_slpa2 = get_communities(filtered, 2016,'label', 1)
labels_slpa3 = get_communities(fil_wo, 2016,'label', 1)

199
150


In [162]:
# labels_modn1 = get_communities(udf, 2016,'modn', 1)
# # labels_modn2 = get_communities(filtered, 2016,'modn', 1)
# labels_modn3 = get_communities(fil_wo, 2016,'modn', 1)

# Spectral Clustering

In [216]:
sc = SpectralClustering(6, affinity='precomputed', n_init=100)

In [217]:
labels1 = sc.fit_predict(adj_mat1)
# labels2 = sc.fit_predict(adj_mat2)
labels3 = sc.fit_predict(adj_mat3)

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:234: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:261: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  "Graph is not fully connected, spectral embedding may not work as expected."
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:234: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:261: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  "Graph is not fully connected, spectral embedding may not work as expected."
/opt/conda/lib/python3.7/site-packages/skl

In [218]:
print("freq1")
unique, counts = np.unique(labels1, return_counts=True)
print(np.asarray((unique, counts)).T)
# print("freq2")
# unique, counts = np.unique(labels2, return_counts=True)
# print(np.asarray((unique, counts)).T)
print("freq3")
unique, counts = np.unique(labels3, return_counts=True)
print(np.asarray((unique, counts)).T)

freq1
[[  0 175]
 [  1   3]
 [  2   6]
 [  3  11]
 [  4   1]
 [  5   3]]
freq2
[[  0  14]
 [  1   3]
 [  2 150]
 [  3   7]
 [  4  24]
 [  5   1]]
freq3
[[  0  22]
 [  1   3]
 [  2 103]
 [  3   7]
 [  4   1]
 [  5  14]]


# Motif-Based Spectral Clustering

In [166]:
!pip install motifcluster

Clustering.py

In [170]:
from motifcluster import motifadjacency as mcmo
from sklearn.cluster import KMeans
from scipy import sparse
from scipy import linalg
from numpy import random as rd

def cluster_spectrum(spectrum, num_clusts):

  vects = spectrum["vects"][:, 1:]
  kmeans_plus_plus = KMeans(n_clusters=num_clusts).fit(vects)
  cluster_assigns = kmeans_plus_plus.labels_

  return cluster_assigns


def run_motif_clustering(adj_mat, motif_name,
                         motif_type="struc",
                         mam_weight_type="unweighted",
                         mam_method="sparse",
                         num_eigs=2,
                         type_lap="comb",
                         num_clusts=2,
                         restrict=True,
                         gr_method="sparse"):


  assert motif_type in ["struc", "func"]
  assert mam_weight_type in ["unweighted", "mean", "product"]
  assert mam_method in ["sparse", "dense"]
  assert isinstance(restrict, bool)
  assert type_lap in ["comb", "rw"]

  spectrum = run_motif_embedding(adj_mat, motif_name, motif_type, mam_weight_type,
                                      mam_method, num_eigs, type_lap, restrict, gr_method)

  cluster_assigns = cluster_spectrum(spectrum, num_clusts)

  ans = {
    "adj_mat": adj_mat,
    "motif_adj_mat": spectrum["motif_adj_mat"],
    "comps": spectrum["comps"],
    "adj_mat_comps": spectrum["adj_mat_comps"],
    "motif_adj_mat_comps": spectrum["motif_adj_mat_comps"],
    "vals": spectrum["vals"],
    "vects": spectrum["vects"],
    "clusts": cluster_assigns
  }

  return ans

spectral.py

In [171]:
"""
Functions relating to spectral methods
are in `motifcluster.spectral`.
"""




def _get_first_eigs(some_mat, num_eigs):

  # check args
  assert num_eigs == np.floor(num_eigs)
  assert num_eigs >= 1

  # get spectrum for many eigs
  if num_eigs >= some_mat.shape[0] - 1:

    # make sure matrix is dense
    if sparse.issparse(some_mat):
      some_mat = some_mat.toarray()

    vals, vects = linalg.eig(some_mat)
    ordering = np.argsort(vals.real)
    vals = vals.real[ordering]
    vects = vects.real[:, ordering]
    vals = vals[0:num_eigs]
    vects = vects[:, 0:num_eigs]

  # get spectrum for few eigs
  else:

    # make sure matrix is sparse
    if not sparse.issparse(some_mat):
      some_mat = sparse.csr_matrix(some_mat, dtype="f")

    vals, vects = sparse.linalg.eigs(some_mat, k=num_eigs, which="SM")
    ordering = np.argsort(vals.real)
    vals = vals.real[ordering]
    vects = vects.real[:, ordering]

  # return a list
  spectrum = {
    "vects": vects,
    "vals": vals
  }

  return spectrum


def build_laplacian(adj_mat, type_lap="rw"):


  # check args
  assert type_lap in ["comb", "rw"]
  n = adj_mat.shape[0]

  if not sparse.issparse(adj_mat):
    adj_mat = sparse.csr_matrix(adj_mat)

  # initialize parameters
  degs_adj_mat = adj_mat.sum(axis=0).reshape(n)

  # combinatorial Laplacian
  if type_lap == "comb":
    degs_matrix = sparse.diags(degs_adj_mat, offsets=[0], shape=(n, n))
    L = degs_matrix - adj_mat

  # random-walk Laplacian
  elif type_lap == "rw":
    assert (degs_adj_mat > 0).all()
    inv_degs_matrix = sparse.diags(1 / degs_adj_mat, offsets=[0], shape=(n, n))
    L = sparse.identity(n) - inv_degs_matrix * adj_mat

  L = sparse.csr_matrix(L)

  return L


def run_laplace_embedding(adj_mat, num_eigs, type_lap="rw"):

  
  # check args
  assert num_eigs == np.floor(num_eigs)
  assert num_eigs >= 1
  assert type_lap in ["comb", "rw"]

  # build and embed Laplacian
  laplacian = build_laplacian(adj_mat, type_lap)
  spectrum = _get_first_eigs(laplacian, num_eigs)

  return spectrum


def run_motif_embedding(adj_mat, motif_name,
                        motif_type="struc",
                        mam_weight_type="unweighted",
                        mam_method="sparse",
                        num_eigs=2,
                        type_lap="rw",
                        restrict=True,
                        gr_method="sparse"):


  # check args
  assert motif_name in get_motif_names()
  assert motif_type in ["struc", "func"]
  assert num_eigs == np.floor(num_eigs)
  assert num_eigs >= 1
  assert mam_weight_type in ["unweighted", "mean", "product"]
  assert mam_method in ["sparse", "dense"]
  assert type_lap in ["comb", "rw"]
  assert isinstance(restrict, bool)
  assert gr_method in ["sparse", "dense"]

  if not sparse.issparse(adj_mat):
    adj_mat = sparse.csr_matrix(adj_mat)

  # build motif adjacency matrix
  motif_adj_mat = mcmo.build_motif_adjacency_matrix(adj_mat, motif_name,
                                                    motif_type, mam_weight_type,
                                                    mam_method)

  if restrict:

    # restrict to largest connected component
    comps = get_largest_component(motif_adj_mat, gr_method)

    adj_mat_comps = adj_mat[comps, :].tocsc()[:, comps]
    adj_mat_comps = sparse.csr_matrix(adj_mat_comps)

    motif_adj_mat_comps = motif_adj_mat[comps, :].tocsc()[:, comps]
    motif_adj_mat_comps = sparse.csr_matrix(motif_adj_mat_comps)

    # Laplace embedding restricted
    spect = run_laplace_embedding(motif_adj_mat_comps, num_eigs, type_lap)

  else:
    comps = None
    adj_mat_comps = None
    motif_adj_mat_comps = None

    # Laplace embedding unrestricted
    spect = run_laplace_embedding(motif_adj_mat, num_eigs, type_lap)


  # return list
  embedding = {
    "adj_mat": adj_mat,
    "motif_adj_mat": motif_adj_mat,
    "comps": comps,
    "adj_mat_comps": adj_mat_comps,
    "motif_adj_mat_comps": motif_adj_mat_comps,
    "vals": spect["vals"],
    "vects": spect["vects"],
  }

  return embedding

utils.py

In [172]:
"""
Assorted utility functions for the motifcluster module
are in `motifcluster.utils`.
"""

import random
import numpy as np
from numpy import random as rd

def _a_b_one(a_mat, b_mat):

  if not sparse.issparse(a_mat):
    a_mat = sparse.csr_matrix(a_mat)

  if not sparse.issparse(b_mat):
    b_mat = sparse.csr_matrix(b_mat)

  n = a_mat.shape[0]
  ones_vec = np.ones(n)
  ans = (a_mat.T.multiply(b_mat @ ones_vec)).T

  return ans


def _a_one_b(a_mat, b_mat):

  if not sparse.issparse(a_mat):
    a_mat = sparse.csr_matrix(a_mat)

  if not sparse.issparse(b_mat):
    b_mat = sparse.csr_matrix(b_mat)

  n = a_mat.shape[0]
  ones_vec = np.ones(n)
  ans = (a_mat.multiply(ones_vec @ b_mat))

  return ans


def _drop0_killdiag(some_mat):

  if sparse.issparse(some_mat):
    I = sparse.identity(some_mat.shape[0])
    ans = some_mat - I.multiply(some_mat)

  else:
    I = np.identity(some_mat.shape[0])
    ans = some_mat - some_mat * I

  return ans


def get_largest_component(adj_mat, gr_method):


  if gr_method == "sparse":

    if not sparse.issparse(adj_mat):
      adj_mat = sparse.csr_matrix(adj_mat)

    gr = nx.from_scipy_sparse_matrix(adj_mat > 0)

  else:

    if isinstance(adj_mat, np.ndarray): # pylint: disable=else-if-used
      gr = nx.from_numpy_array(1 * np.array(adj_mat > 0))

    else:
      gr = nx.from_numpy_array(1 * (adj_mat > 0).toarray())

  verts_to_keep = max(nx.connected_components(gr), key=len)
  verts_to_keep = sorted(verts_to_keep)

  return verts_to_keep


def get_motif_names():

  motif_names = ["Ms", "Md"]

  for i in range(1, 14):
    motif_name = "M" + str(i)
    motif_names = motif_names + [motif_name]

  motif_names = motif_names + ["Mcoll"] + ["Mexpa"]

  return motif_names


def _random_sparse_matrix(m, n, p, sample_weight_type="constant", w=1):

  mn = m * n

  # number of nonzero entries
  k = rd.binomial(mn, p)

  # indices of nonzero entries
  zs = k * [0]
  inds = random.sample(range(mn), k)

  # values to go in matrix
  if sample_weight_type == "constant":
    vals = np.full(k, w)

  elif sample_weight_type == "poisson":
    vals = rd.poisson(w, size=k)

  else:
    vals = np.ones(k)

  # create small matrix
  if mn <= 1e4:
    ans = np.zeros(mn)
    ans[np.array(inds, dtype=int)] = np.array(vals, dtype=int)
    ans = ans.reshape((m, n))

  # create large matrix
  else:
    ans = sparse.csc_matrix((vals, (inds, zs)), shape=(mn, 1))
    ans = ans.reshape((m, n))

  return ans

In [175]:
names = get_motif_names()
names

['Ms',
 'Md',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'M10',
 'M11',
 'M12',
 'M13',
 'Mcoll',
 'Mexpa']

In [176]:
motif_labels = []

In [239]:
motif_algo1 = run_motif_clustering(adj_mat1, "M1",num_eigs=71,num_clusts=6,restrict=False)
# print(motif_algo)
# print(len(motif_algo['comps']))
print(motif_algo1['vects'])
print(motif_algo1['comps'])
print(len(motif_algo1['clusts']))
print(motif_algo1['clusts'])
print_freq_table(motif_algo1['clusts'])
# print(len(motif_algo['comps']))
# print(len(motif_algo['comps']))
# print(len(motif_algo['comps']))
# print_freq_table(motif_algo['clusts'])
# data_array = motif_algo['comps']
# motif_csv = pd.DataFrame(data = data_array, columns=['Node_number'])
# motif_csv['Community_id'] = motif_algo['clusts']

[[ 2.17079224e-02 -3.89820374e-02  1.72485601e-02 ... -2.30325897e-02
  -1.46120404e-03  9.26258575e-04]
 [-7.82776800e-02 -5.67689372e-02  2.04027774e-02 ...  9.07408568e-02
   1.64219775e-02 -7.31400241e-02]
 [ 5.05234403e-03 -1.31188165e-02 -1.07827056e-01 ... -1.81099389e-16
   1.61361660e-17  3.34487057e-18]
 ...
 [-3.28608639e-02 -5.42667014e-02  2.13048520e-02 ... -6.61661808e-02
   1.94951251e-02  1.93568654e-02]
 [ 1.49706135e-01  1.31035223e-01 -4.84181874e-02 ... -1.51089805e-01
  -2.49856288e-01  3.79594507e-02]
 [ 2.96095314e-02 -4.12873353e-02  1.85522501e-02 ... -1.99858285e-02
   2.08211406e-02 -4.88986499e-02]]
None
199
[0 1 0 0 0 2 0 0 1 4 1 3 3 0 3 0 5 0 0 1 0 0 0 1 2 0 4 3 2 0 0 0 0 2 0 2 0
 0 5 0 4 0 4 0 0 0 2 0 4 0 0 3 0 0 3 0 0 0 0 0 4 0 0 4 2 0 3 0 0 0 0 2 0 0
 0 0 2 0 0 0 4 0 0 0 0 0 0 0 0 4 3 0 0 4 0 1 2 0 0 3 0 0 0 0 1 4 0 0 1 0 3
 0 0 0 0 0 1 5 0 0 0 0 0 0 4 0 0 0 0 1 0 3 0 0 0 3 0 2 0 0 0 3 3 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 4 0 1 0 0 0 4 0 0 0 0 0 3 0 0 4 4 0 

In [240]:
motif_algo3 = run_motif_clustering(adj_mat3, "M1",num_eigs=71,num_clusts=6,restrict=False)
# print(motif_algo)
# print(len(motif_algo['comps']))
print(motif_algo3['vects'])
print(motif_algo3['comps'])
print(len(motif_algo3['clusts']))
print(motif_algo3['clusts'])
print_freq_table(motif_algo3['clusts'])

[[ 2.31603645e-01  1.65400873e-01  3.67562310e-02 ... -1.43771165e-01
  -4.99311167e-02  1.33175857e-01]
 [ 7.36655775e-02  2.22981973e-02  6.29459726e-03 ... -9.04160441e-02
  -3.43975471e-02  1.72753728e-02]
 [-2.54526138e-02 -7.80286064e-02 -4.74074537e-05 ...  1.92109316e-11
   3.65176754e-11  9.04666502e-02]
 ...
 [ 9.43426833e-02 -2.06481128e-01 -2.07085437e-02 ...  1.44591740e-01
  -2.37045647e-01 -1.86847903e-01]
 [ 1.19785821e-01  1.65298536e-02  3.71584250e-03 ... -7.57765390e-02
   1.70280192e-01  1.50914019e-02]
 [ 6.87355526e-02  1.60223907e-02  6.62388920e-03 ...  6.74861324e-02
   1.23135245e-01  1.03880926e-02]]
None
150
[5 0 0 5 3 0 0 0 4 4 3 0 0 1 0 0 0 0 2 0 0 0 0 5 4 1 0 0 0 3 0 0 0 0 0 0 5
 0 0 0 0 0 0 0 0 0 0 5 4 0 0 0 0 2 0 0 0 4 0 0 0 4 3 0 0 0 0 0 0 0 0 0 0 0
 5 0 0 0 0 0 0 5 0 1 0 5 0 0 0 0 0 0 0 0 0 0 2 0 0 5 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 3 0 0 0 4 0 0 4 0 0 0 4 0 0 0 0 0 3 0 3 0 0 0 0 1 0 0 4 0 0 5 5 0 4
 5 5]
[[  0 112]
 [  1   4]
 [  2   3]
 [  3   7]
 [  4

# Block Models


In [183]:
!pip3 install sparsebm

In [184]:
!pip3 install cupy

  Using cached cupy-11.4.0.tar.gz (1.8 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [63 lines of output]
      
      -------- Configuring Module: cuda --------
      cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
      /tmp/tmpmwpmqdk_/a.cpp:1:10: fatal error: cublas_v2.h: No such file or directory
          1 | #include <cublas_v2.h>
            |          ^~~~~~~~~~~~~
      compilation terminated.
      command 'gcc' failed with exit status 1
      cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
      /tmp/tmpr28gv2cy/a.cpp:2:18: fatal error: cuda_runtime_api.h: No such file or directory
          2 |         #include <cuda_runtime_api.h>
            |                  ^~~~~~~~~~~~~~~~~~~~
      compilation terminated.
      ******************************************

In [185]:
def print_freq_table(array):
    unique, counts = np.unique(array, return_counts=True)
    print(np.asarray((unique, counts)).T)

In [186]:
from sparsebm import generate_SBM_dataset
from sparsebm import SBM

In [187]:
def get_fitted_model(adj_mat, num_clusters):
    # A number of classes must be specify. Otherwise see model selection.
    model = SBM(num_clusters)
    model.fit(adj_mat, symmetric=False)
#     print("Labels:", model.labels)
    return model

# from sparsebm.utils import ARI
# ari = ARI(cluster_indicator.argmax(1), model.labels)
# print("Adjusted Rand index is {:.2f}".format(ari))

In [188]:
# for i in range(4, 20):
#     mod = get_fitted_model(i)
#     print_freq_table(mod.labels)

In [221]:
sparse_model1 = get_fitted_model(adj_mat1, 6)
# sparse_model2 = get_fitted_model(adj_mat2, 8)
sparse_model3 = get_fitted_model(adj_mat3, 6)

---------- START RANDOM INITIALIZATIONS ---------- 
40 of 100 Initializations:  [ 40% ] |#####         | [ Elapsed Time: 0:00:00 ] /opt/conda/lib/python3.7/site-packages/sparsebm/sbm.py:436: RuntimeWarning: invalid value encountered in log
  - tau @ np.log(1 - pi)
/opt/conda/lib/python3.7/site-packages/sparsebm/sbm.py:447: RuntimeWarning: invalid value encountered in log
  - tau @ np.log(1 - pi.T)
100 of 100 Initializations:  [100% ] |#############| [ Elapsed Time: 0:00:01 ] 
---------- START TRAINING BEST INITIALIZATIONS ---------- 
10 of 10 Runs:  [100% ] |##########################| [ Elapsed Time: 0:00:00 ] 
---------- START RANDOM INITIALIZATIONS ---------- 
100 of 100 Initializations:  [100% ] |#############| [ Elapsed Time: 0:00:01 ] 
---------- START TRAINING BEST INITIALIZATIONS ---------- 
10 of 10 Runs:  [100% ] |##########################| [ Elapsed Time: 0:00:25 ] 


In [222]:
print("freq1")
unique, counts = np.unique(sparse_model1.labels, return_counts=True)
print(np.asarray((unique, counts)).T)
print("freq3")
unique, counts = np.unique(sparse_model3.labels, return_counts=True)
print(np.asarray((unique, counts)).T)

freq1
[[ 0  2]
 [ 1 36]
 [ 3  1]
 [ 4 90]
 [ 5 70]]
freq2
[[ 0 49]
 [ 3 70]
 [ 4 29]
 [ 5 51]]
freq3
[[ 0 25]
 [ 1 53]
 [ 5 72]]


# Clustering Metrics

link modularity (LM) (Nicosia et al., 2009), internal edge density (IED) (Radicchi et al., 2004), average internal degree (AID) (Radicchi et al., 2004), cut ratio (CR) (Fortunato, 2010), and Z-modularity (ZM) (Miyauchi and Kawase, 2016)

In [194]:
!pip install cdlib

In [195]:
from cdlib import evaluation
from cdlib import NodeClustering

In [196]:
unique, counts = np.unique(labels1, return_counts=True)
print(unique)
print(counts)

[0 1 2 3 4 5 6 7]
[140   1   4   3   3   8   6  34]


In [197]:
labels1

array([0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 3, 0, 0,
       5, 7, 4, 5, 0, 7, 7, 0, 0, 4, 0, 5, 7, 0, 0, 6, 5, 6, 0, 7, 0, 0,
       0, 0, 7, 0, 6, 0, 7, 7, 0, 7, 7, 0, 0, 0, 7, 7, 0, 0, 7, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 7, 0, 0,
       0, 0, 0, 7, 7, 0, 0, 0, 7, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 5,
       0, 3, 0, 0, 0, 5, 0, 0, 0, 7, 0, 1, 6, 0, 0, 0, 0, 0, 5, 0, 0, 0,
       0, 0, 7, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 7, 7, 0, 0, 0, 7, 7,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 7, 0, 6, 0, 0, 6, 3,
       7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 7, 0,
       0], dtype=int32)

In [198]:
def get_comms_from_list(l):
    comms = []
    unique, counts = np.unique(l, return_counts=True)
    comms = [[y for y in range(len(l)) if l[y]==x] for x in unique]
    return comms
# get_comms_from_list(labels1)
    

In [199]:
def get_comms_from_vertex_clustering(vc):
    temp_arr = np.zeros(vc.n, dtype='int')
    for i in range(len(vc)):
        for j in range(len(vc[i])):
            temp_arr[vc[i][j]] = i
    unique, counts = np.unique(temp_arr, return_counts=True)
    comms = [[y for y in range(len(temp_arr)) if temp_arr[y]==x] for x in unique]
    return comms

In [223]:
comms_spectral_g1 = NodeClustering(get_comms_from_list(labels1), graph=None, method_name="spectral")
# comms_spectral_g2 = NodeClustering(get_comms_from_list(labels2), graph=None, method_name="spectral")
comms_spectral_g3 = NodeClustering(get_comms_from_list(labels3), graph=None, method_name="spectral")
# comms_spectral_g1.communities

In [224]:
comms_sparseBM1 = NodeClustering(get_comms_from_list(sparse_model1.labels), graph=None, method_name="sparseBM")
# comms_sparseBM2 = NodeClustering(get_comms_from_list(sparse_model2.labels), graph=None, method_name="sparseBM")
comms_sparseBM3 = NodeClustering(get_comms_from_list(sparse_model3.labels), graph=None, method_name="sparseBM")
# len(comms_sparseBM.communities)

In [241]:
comms_motif_g1 = NodeClustering(get_comms_from_list(motif_algo1['clusts']), graph=None, method_name="motif")
comms_motif_g3 = NodeClustering(get_comms_from_list(motif_algo3['clusts']), graph=None, method_name="motif")

In [203]:
comms_info_g1 = NodeClustering(get_comms_from_vertex_clustering(labels_info1), graph=None, method_name="infomap")
comms_info_g3 = NodeClustering(get_comms_from_vertex_clustering(labels_info3), graph=None, method_name="infomap")

In [204]:
comms_slpa_g1 = NodeClustering(get_comms_from_vertex_clustering(labels_slpa1), graph=None, method_name="slpa")
comms_slpa_g3 = NodeClustering(get_comms_from_vertex_clustering(labels_slpa3), graph=None, method_name="slpa")

In [214]:
# print(labels_info1)
# print(labels_info3)

Clustering with 199 elements and 6 clusters
[0] 0, 12, 120, 133, 186
[1] 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 20, 22, 23, 24,
    25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
    43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
    62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
    80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
    98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 112, 113, 114,
    115, 116, 117, 118, 119, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
    132, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 145, 146, 148, 149,
    150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
    165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 180,
    181, 182, 184, 185, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197,
    198
[2] 19, 111, 121, 175
[3] 21, 44, 140, 147
[4] 104
[5] 183
Clusterin

In [227]:
print("SPECTRAL")
print("Link modularity = ")
mod_spectral_g1 = evaluation.link_modularity(G1, comms_spectral_g1)
mod_spectral_g3 = evaluation.link_modularity(G3, comms_spectral_g3)
print(mod_spectral_g1.score)
print(mod_spectral_g3.score)
print("Internal_edge_density = ")
mod_spectral_g1 = evaluation.internal_edge_density(G1, comms_spectral_g1)
mod_spectral_g3 = evaluation.internal_edge_density(G3, comms_spectral_g3)
print(mod_spectral_g1.score)
print(mod_spectral_g3.score)
print("Average_internal_degree = ")
mod_spectral_g1 = evaluation.average_internal_degree(G1, comms_spectral_g1)
mod_spectral_g3 = evaluation.average_internal_degree(G3, comms_spectral_g3)
print(mod_spectral_g1.score)
print(mod_spectral_g3.score)
print("Cut ratio = ")
mod_spectral_g1 = evaluation.cut_ratio(G1, comms_spectral_g1)
mod_spectral_g3 = evaluation.cut_ratio(G3, comms_spectral_g3)
print(mod_spectral_g1.score)
print(mod_spectral_g3.score)
print("Z modularity = ")
mod_spectral_g1 = evaluation.z_modularity(G1, comms_spectral_g1)
mod_spectral_g3 = evaluation.z_modularity(G3, comms_spectral_g3)
print(mod_spectral_g1.score)
print(mod_spectral_g3.score)

SPECTRAL
Link modularity = 
0.06134860091640554
0.04187841110788913
Internal_edge_density = 
0.383446285515251
0.35842721907827557
Average_internal_degree = 
7.678239538239538
8.61729640370417
Cut ratio = 
0.052314726545162636
0.09712195636803785
Z modularity = 
0.02828339093133384
0.06141253769979121


In [243]:
print("MOTIF")
print("Link modularity = ")
mod_motif_g1 = evaluation.link_modularity(G1, comms_motif_g1)
mod_motif_g3 = evaluation.link_modularity(G3, comms_motif_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)
print("Internal_edge_density = ")
mod_motif_g1 = evaluation.internal_edge_density(G1, comms_motif_g1)
mod_motif_g3 = evaluation.internal_edge_density(G3, comms_motif_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)
print("Average_internal_degree = ")
mod_motif_g1 = evaluation.average_internal_degree(G1, comms_motif_g1)
mod_motif_g3 = evaluation.average_internal_degree(G3, comms_motif_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)
print("Cut ratio = ")
mod_motif_g1 = evaluation.cut_ratio(G1, comms_motif_g1)
mod_motif_g3 = evaluation.cut_ratio(G3, comms_motif_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)
print("Z modularity = ")
mod_motif_g1 = evaluation.z_modularity(G1, comms_motif_g1)
mod_motif_g3 = evaluation.z_modularity(G3, comms_motif_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

MOTIF
Link modularity = 
0.04301765782021004
0.046677672090599746
Internal_edge_density = 
0.1047348467896559
0.1326939951939952
Average_internal_degree = 
7.0699295769422
6.995233932733933
Cut ratio = 
0.060734506826055586
0.09607427123728829
Z modularity = 
0.0010442955941301023
-0.006763810827385698


In [231]:
print("BLOCK")
print("Link modularity = ")
mod_motif_g1 = evaluation.link_modularity(G1, comms_sparseBM1)
mod_motif_g3 = evaluation.link_modularity(G3, comms_sparseBM3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)
print("Internal_edge_density = ")
mod_motif_g1 = evaluation.internal_edge_density(G1, comms_sparseBM1)
mod_motif_g3 = evaluation.internal_edge_density(G3, comms_sparseBM3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

print("Average_internal_degree = ")
mod_motif_g1 = evaluation.average_internal_degree(G1, comms_sparseBM1)
mod_motif_g3 = evaluation.average_internal_degree(G3, comms_sparseBM3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

print("Cut ratio = ")
mod_motif_g1 = evaluation.cut_ratio(G1, comms_sparseBM1)
mod_motif_g3 = evaluation.cut_ratio(G3, comms_sparseBM3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

print("Z modularity = ")
mod_motif_g1 = evaluation.z_modularity(G1, comms_sparseBM1)
mod_motif_g3 = evaluation.z_modularity(G3, comms_sparseBM3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

BLOCK
Link modularity = 
0.018040701647349413
0.021045795815901474
Internal_edge_density = 
0.11475670939276217
0.2759580500797615
Average_internal_degree = 
6.146349206349207
12.01728860936408
Cut ratio = 
0.07307931061792741
0.168908187793677
Z modularity = 
-0.20066300443386792
-0.14849271195197739


In [238]:
print("INFO")
print("Link modularity = ")
mod_motif_g1 = evaluation.link_modularity(G1, comms_info_g1)
mod_motif_g3 = evaluation.link_modularity(G3, comms_info_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)
print("Internal_edge_density = ")
mod_motif_g1 = evaluation.internal_edge_density(G1, comms_info_g1)
mod_motif_g3 = evaluation.internal_edge_density(G3, comms_info_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

print("Average_internal_degree = ")
mod_motif_g1 = evaluation.average_internal_degree(G1, comms_info_g1)
mod_motif_g3 = evaluation.average_internal_degree(G3, comms_info_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

print("Cut ratio = ")
mod_motif_g1 = evaluation.cut_ratio(G1, comms_info_g1)
mod_motif_g3 = evaluation.cut_ratio(G3, comms_info_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

print("Z modularity = ")
mod_motif_g1 = evaluation.z_modularity(G1, comms_info_g1)
mod_motif_g3 = evaluation.z_modularity(G3, comms_info_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

INFO
Link modularity = 
0.062707059224118
0.07016770062004962
Internal_edge_density = 
0.3167735804229033
0.4000236183278224
Average_internal_degree = 
7.0862318840579706
22.503424657534246
Cut ratio = 
0.07561564512483383
0.03424657534246575
Z modularity = 
0.008646609835871092
0.014897222682149353


In [210]:
print("LABEL (SLPA)")
print("Link modularity = ")
mod_motif_g1 = evaluation.link_modularity(G1, comms_slpa_g1)
mod_motif_g3 = evaluation.link_modularity(G3, comms_slpa_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)
print("Internal_edge_density = ")
mod_motif_g1 = evaluation.internal_edge_density(G1, comms_slpa_g1)
mod_motif_g3 = evaluation.internal_edge_density(G3, comms_slpa_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)
print("Average_internal_degree = ")
mod_motif_g1 = evaluation.average_internal_degree(G1, comms_slpa_g1)
mod_motif_g3 = evaluation.average_internal_degree(G3, comms_slpa_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

print("Cut ratio = ")
mod_motif_g1 = evaluation.cut_ratio(G1, comms_slpa_g1)
mod_motif_g3 = evaluation.cut_ratio(G3, comms_slpa_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

print("Z modularity = ")
mod_motif_g1 = evaluation.z_modularity(G1, comms_slpa_g1)
mod_motif_g3 = evaluation.z_modularity(G3, comms_slpa_g3)
print(mod_motif_g1.score)
print(mod_motif_g3.score)

LABEL (SLPA)
Link modularity = 
0.052193147136671066
0.0614775252239397
Internal_edge_density = 
0.20165576655372572
0.337850825410634
Average_internal_degree = 
1.4033976833976833
4.396308954203691
Cut ratio = 
0.07698141312570304
0.12673727585916097
Z modularity = 
-0.007064326352392115
0.004004485761264681
